# Outage Duration vs Distance to Feature

Goal: Create a visualization that shows the relationship between outage duration and distance to feature:

Will use 3 features:
  - Nearest Substation
  - Nearest Populated Place
  - Nearest Populated Place with at least 100 people

In [ ]:
import sys
import plotly.graph_objects as go

# Allow importing from parent directory by temporarily moving the CWD up one level
# Very hacky, but there literally isn't a simpler way (in Jupyter)
sys.path.append("..")
from common import get_dataframe_from_pipeline

outages = get_dataframe_from_pipeline("../pipeline/3.csv.gz")
# Drop the path back down after import
sys.path.pop()

outages['duration'] = outages['dateOn'] - outages['dateOff']

In [ ]:
fig = go.Figure(
  layout=go.Layout(
    title={
      "text": "Mean Power Outage Length vs Distance to Various Locations in BC",
      "x": 0.5,
    },
    xaxis_title="Rounded Distance (km)",
    yaxis_title="Outage Length (hours)",
    xaxis_type="log",  # Set x-axis to logarithmic scale
    yaxis_type="log",  # Set y-axis to logarithmic scale
  )
)

outages['subBin'] = outages['outageToSubstationDistance'].round(0)
subGrouped = outages.groupby(by='subBin').aggregate(
  {
    'duration': lambda x: x.mean(),
  }
).reset_index()

fig.add_bar(
  x=subGrouped['subBin'],
  y=subGrouped['duration'].dt.total_seconds() / 3600,
  marker_color='blue',
  showlegend=True,
  name="Distance to Substation"
)

outages['placeBin'] = outages['outageToPopulatedPlaceDistance'].round(0)
placeGroupedGrouped = outages.groupby(by='placeBin').aggregate(
  {
    'duration': lambda x: x.mean(),
  }
).reset_index()

fig.add_bar(
  x=placeGroupedGrouped['placeBin'],
  y=placeGroupedGrouped['duration'].dt.total_seconds() / 3600,
  marker_color='red',
  showlegend=True,
  name="Distance to Populated Place",
)

outages['urbanAreaBin'] = outages['outageToUrbanAreaDistance'].round(0)
urbanAreaGrouped = outages.groupby(by='urbanAreaBin').aggregate(
  {
    'duration': lambda x: x.mean(),
  }
).reset_index()

fig.add_bar(
  x=urbanAreaGrouped['urbanAreaBin'],
  y=urbanAreaGrouped['duration'].dt.total_seconds() / 3600,
  marker_color='green',
  showlegend=True,
  name="Distance to Urban Area",
)

fig

In [ ]:
import os

# Save the map
# Make directory if it doesn't exist
if not os.path.exists("html"):
  os.makedirs("html")

fig.write_html("html/outage_length_vs_distance.html", include_plotlyjs="cdn")